In [159]:
#!pip install automathon

In [160]:
"""Convert regular expression to NFA."""
import json
import graphviz

def validate_reg(reg):
    #check if the input regular expression is valid
    if reg=="":
        return False
    if reg[0]=="*" or reg[0]=="+" or reg[0]=="?"  :
        #example "*abc" is invalid
        return False
    # if reg[-1]=="|":
    #     #example "a|b|c|"" is invalid
    #     return False
    for i in range(0,len(reg)-1):
        if reg[i]=="*":##error reg[i]=="+" when 3  w kaman reg[i]=="?
            if reg[i+1]==reg[i]:
                #example "a|b**c" is invalid
                return False
        if i<len(reg)-2 and ((reg[i]=="+" and reg[i+1]=="+" and reg[i+2]=="+")or(reg[i]=="?" and reg[i+1]=="?" and reg[i+2]=="?")):
            #example "a+++" or "a???" is invalid
            return False
        if reg[i]=="+" and reg[i+1]=="*" :
            #example "a+*" is invalid
            return False
        if reg[i]=="?" and reg[i+1]=="*" :
            #example "a?*" is invalid
            return False
        if reg[i]=="|" :
            if reg[i+1]=="*" or reg[i]=="?" or reg[i+1]=="+" or reg[i+1]==")" or reg[i+1]=="]":
                #example "a|b|+c" or "a|b|+*" is invalid
                return False

        if reg[i]=="-":#TODO checkkkkk if all alphanumeric
            if i>0 and reg[i+1].isalnum() and reg[i-1].isalnum():
                continue
            return False
        if reg[i]=="[":
            if reg[i+1]=="]" or reg[i+1]=="|":#[] invalid
                return False
    return True
def validate_brackets(reg):
    stack=[]
    for char in reg:
        if char=="(":
            stack.append(char)
        elif char==")":
            #stack[-1] is the last element in the stack
            while stack and stack[-1]!="(":
                stack.pop()
            if stack:
                stack.pop()
            else:
                return "unbalanced parentheses"
        elif char=="[":
            stack.append(char)
        elif char=="]":
            #stack[-1] is the last element in the stack
            while stack and stack[-1]!="[":
                stack.pop()
            if stack:
                stack.pop()
            else:
                return "unbalanced parentheses"
    if stack:
        return "unbalanced parentheses"

    return "balanced parentheses"
#start with adding special character to represent Anding
def regex_anding(reg):
    correct_reg=[]
    range_str=""
    is_range=False
    last_is_put=False
    first_is_operator_and_already_put=False
    # take care this loop is from 0 to len(reg)-2 so concentarte to know how we handle the last character
    for i in range(0,len(reg)-1):
        if is_range:
            if reg[i]=="]":
                is_range=False
                correct_reg.append(range_str)
                range_str=""
            else:
                range_str+=reg[i]
                continue
        if reg[i]=="|" or reg[i+1]=="|":
            if not first_is_operator_and_already_put:
                correct_reg.append(reg[i])
            else:
                first_is_operator_and_already_put=False
            continue
        elif reg[i+1]=="?" or reg[i+1]=="*" or reg[i+1]=="+":
            if not first_is_operator_and_already_put:
                correct_reg.append(reg[i])
            else:
                first_is_operator_and_already_put=False
            correct_reg.append(reg[i+1])
            # this is the only case where we put TWO chars together
            # so, if we reached the end of the regular expression,
            # we dont want to put the last character (which is always an operator as we are in this elif) again
            # as it will be put in the last if condition
            # so we set last_is_put to True to avoid putting it again
            if i==len(reg)-2:
                last_is_put=True
                break
            #TODO check if the next character is an operator if yes dont put anding
            if i<len(reg)-2 and (reg[i+2]=="*" or reg[i+2]=="+" or reg[i+2]=="?" or reg[i+2]=="|"):#example "a*?" we dont want to put anding
                first_is_operator_and_already_put=True #if we have multiple operators together we dont want to put them twice each
                continue
            correct_reg.append("×")
            continue
        # elif reg[i]=="]":
        #     if reg[i+1].isalnum() or reg[i+1]=="[" or reg[i+1]=="(":
        #         #we will not put the "]" in the correct_reg as it is already put as a string (range_str) in the correct_reg
        #         correct_reg.append("×")
        #         continue
        elif reg[i].isalnum() or reg[i]=="]" or reg[i]==")":
            if reg[i+1].isalnum() or reg[i+1]=="[" or reg[i+1]=="(":
                correct_reg.append(reg[i])
                correct_reg.append("×")
                continue
        elif reg[i]=="[" :
            correct_reg.append(reg[i])
            range_str=""
            is_range=True
            continue
        if reg[i]=="?" or reg[i]=="*" or reg[i]=="+":
            continue
        else:
            correct_reg.append(reg[i])
    # put the last character in the regular expression
    # only if last_is_put is False
    if i==len(reg)-2 and not last_is_put:
        if is_range:#this means that the regex was ending with the "]" example 1[a-z] this means that we didnot put the range_str in the correct_reg
                    #as is_range is still True
            correct_reg.append(range_str)
        correct_reg.append(reg[i+1])
    return correct_reg #the result could be ['[', 'a-z_', ']', '[', 'a-z0-9_', ']', '*'] then we want to concat the range in one string (function concat_range)

def concat_range(correct_reg):
    reg=[]
    range_str=""
    is_range=False
    for i in range(0,len(correct_reg)):
        if correct_reg[i]=="]":
            continue
        if is_range:
            range_str="["+correct_reg[i]+"]"
            # check that something like [a-c-f] does not exist
            # so check that the distance between the two "-" is 2 at least. so [a-ce-k] is valid
            indices_of_a = [p for p, char in enumerate(correct_reg[i]) if char == "-"]
            for j in range(0,len(indices_of_a)-1):
                if indices_of_a[j+1]-indices_of_a[j]<3:
                    return ["%"]#just to mark that the regular expression is invalid
            if correct_reg[i+1]!="]":
                return ["%"]#just to mark that the regular expression is invalid
            reg.append(range_str)
            is_range=False
        elif correct_reg[i]=="[" :
            is_range=True
        else:
            reg.append(correct_reg[i])
    return reg


def remove_extra_anding(reg):
    correct_regex=[]
    for i in range(0, len(reg)-1):
        if (reg[i]=="×" and reg[i+1]==")"):
            continue
        correct_regex.append(reg[i])
    correct_regex.append(reg[len(reg)-1])
    return correct_regex

def precedence(operator):
    if operator=="*":
        return 5
    elif operator=="+":
        return 4
    elif operator=="?":
        return 3
    elif operator=="×":
        return 2
    elif operator=="|":
        return 1
    else:
        return -1
def infix_to_postfix(infix):
    operators=["*","+","?","×","|"]
    stack=[]
    postfix=[]
    for char in infix:
        # if char.isalnum() or char=="×":
        #     postfix+=char
        if char=="(":
            stack.append(char)
        elif char==")":
            #stack[-1] is the last element in the stack
            while stack and stack[-1]!="(":
                postfix.append(stack.pop())
            if stack:
                stack.pop()
            else:
                return "unbalanced parentheses"
        elif char in operators:
            while stack and precedence(char)<=precedence(stack[-1]):
                postfix.append(stack.pop())
            stack.append(char)
        else:
            #if char is a alphanumeric character or a range string example "[a-z]"
            postfix.append(char)
    while stack:
        if stack[-1]=="(":
            return "unbalanced parentheses"
        postfix.append(stack.pop())
    return postfix


#convert the postfix to NFA

class State:
    def __init__(self,label):
        self.label=label
        self.transitions=dict()
        self.epsilon_closure=set()
class NFA:
    def __init__(self,start:State,accept:State):
        self.start=start
        self.accept=accept
def concat(nfa1:NFA,nfa2:NFA):
    #create new edge between the accept state of nfa1 and the start state of nfa2
    nfa1.accept.epsilon_closure.add(nfa2.start.label)
    # create new nfa
    new_nfa=NFA(nfa1.start,nfa2.accept)
    return new_nfa

def oring(nfa1:NFA,nfa2:NFA,index):
    #create new start state
    new_start_state=State(index)
    #create new accept state
    new_accept_state=State(index+1)
    #create new edge between the new start state and the start state of nfa1
    new_start_state.epsilon_closure.add(nfa1.start.label)
    #create new edge between the new start state and the start state of nfa2
    new_start_state.epsilon_closure.add(nfa2.start.label)
    #create new edge between the accept state of nfa1 and the new accept state
    nfa1.accept.epsilon_closure.add(new_accept_state.label)
    #create new edge between the accept state of nfa2 and the new accept state
    nfa2.accept.epsilon_closure.add(new_accept_state.label)
    #create new nfa
    new_nfa=NFA(new_start_state,new_accept_state)
    new_states=[new_start_state,new_accept_state]
    return new_nfa,new_states

def zero_or_more(nfa:NFA,index):
    #create new start state
    new_start_state=State(index)
    #create new accept state
    new_accept_state=State(index+1)
    #create new edge between the new start state and the start state of nfa
    new_start_state.epsilon_closure.add(nfa.start.label)
    #create new edge between the new start state and the new accept state
    new_start_state.epsilon_closure.add(new_accept_state.label)
    #create new edge between the accept state of nfa and the new accept state
    nfa.accept.epsilon_closure.add(new_accept_state.label)
    #create new edge between the accept state of nfa and the start state of nfa
    nfa.accept.epsilon_closure.add(new_start_state.label)#(nfa.start.label)
    #create new nfa
    new_nfa=NFA(new_start_state,new_accept_state)
    new_states=[new_start_state,new_accept_state]
    return new_nfa,new_states

def one_or_more(nfa:NFA,index):
    #create new start state
    new_start_state=State(index)
    #create new accept state
    new_accept_state=State(index+1)
    #create new edge between the new start state and the start state of nfa
    new_start_state.epsilon_closure.add(nfa.start.label)
    #create new edge between the accept state of nfa and the new accept state
    nfa.accept.epsilon_closure.add(new_accept_state.label)
    #create new edge between the accept state of nfa and the start state of nfa
    nfa.accept.epsilon_closure.add(new_start_state.label)#(nfa.start.label)
    #create new nfa
    new_nfa=NFA(new_start_state,new_accept_state)
    new_states=[new_start_state,new_accept_state]
    return new_nfa,new_states

def zero_or_one(nfa:NFA,index):
    #create new start state
    new_start_state=State(index)
    #create new accept state
    new_accept_state=State(index+1)
    #create new edge between the new start state and the start state of nfa
    new_start_state.epsilon_closure.add(nfa.start.label)
    #create new edge between the new start state and the new accept state
    new_start_state.epsilon_closure.add(new_accept_state.label)
    #create new edge between the accept state of nfa and the new accept state
    nfa.accept.epsilon_closure.add(new_accept_state.label)
    #create new nfa
    new_nfa=NFA(new_start_state,new_accept_state)
    new_states=[new_start_state,new_accept_state]
    return new_nfa,new_states

def create_nfa(char,index):
    #create new start state
    start_state=State(index)
    #create new accept state
    accept_state=State(index+1)
    #create new edge between the start state and the accept state
    start_state.transitions[char]=accept_state.label
    #create new nfa
    new_nfa=NFA(start_state,accept_state)
    new_states=[start_state,accept_state]
    return new_nfa,new_states



def postfix_to_nfa(postfix):
    nfa_stack=[]
    all_states=[]
    index=0
    for char in postfix:
        if char=="*":
            nfa,new_states=zero_or_more(nfa_stack.pop(),index)
            nfa_stack.append(nfa)
            all_states.extend(new_states)
            index+=2
        elif char=="+":
            nfa,new_states=one_or_more(nfa_stack.pop(),index)
            nfa_stack.append(nfa)
            all_states.extend(new_states)
            index+=2
        elif char=="?":
            nfa,new_states=zero_or_one(nfa_stack.pop(),index)
            nfa_stack.append(nfa)
            all_states.extend(new_states)
            index+=2
        elif char=="×":
            operand2=nfa_stack.pop()
            operand1=nfa_stack.pop()
            nfa=concat(operand1,operand2)
            nfa_stack.append(nfa)
        elif char=="|":
            operand2=nfa_stack.pop()
            operand1=nfa_stack.pop()
            nfa,new_states=oring(operand1,operand2,index)
            nfa_stack.append(nfa)
            all_states.extend(new_states)
            index+=2
        else:
            nfa,new_states=create_nfa(char,index)
            nfa_stack.append(nfa)
            all_states.extend(new_states)
            index+=2
    return nfa_stack.pop(),all_states

def create_dict_for_jason(nfa,all_states):
    nfa_dict=dict()
    nfa_dict["startingState"]="S"+str(nfa.start.label)
    for state in all_states:
        #the state info to be added must be like that
        #"S0": { "isTerminatingState": true, "a": "S0", "b": "S1"}
        #where a, b are the transitions
        state_info=dict()

        if state==nfa.accept:
            state_info["isTerminatingState"]=True
        else:
            state_info["isTerminatingState"]=False
        for char in state.transitions:
            state_info[char]="S"+str(state.transitions[char])

        if len(state.epsilon_closure)>0:
            state_info["epsilon"]=[]
            for epsilon_state in state.epsilon_closure:
                state_info["epsilon"].append("S"+str(epsilon_state))
        nfa_dict["S"+str(state.label)]=state_info

    return nfa_dict

def create_json(nfa_dict):
    with open("NFA.json","w") as file:
        json.dump(nfa_dict,file,indent=4)
    print("json file created successfully")

# Draw the NFA graph using Graphviz

def draw_nfa(json_filename):
    # Load the NFA data from the JSON file
    with open(json_filename, 'r') as json_file:
        nfa_data = json.load(json_file)

    # Create a new Graphviz graph
    graph = graphviz.Digraph(format='png', name='NFA_graph', graph_attr={'rankdir': 'LR'})
    # graph = pydot.Dot("NFA_graph",graph_type='digraph')
    # node=pydot.Node("S"+str(nfa_data['startingState']),label="S"+str(nfa_data['startingState']),shape='circle')

    start_state=None
    # Add nodes for each state
    for state, state_data in nfa_data.items():
        if state == 'startingState':
            graph.node(state_data, label=state_data, shape= 'circle')
            start_state=state_data
            #add an edge to point to start state
            graph.node('None', label='', style='invisible')
            graph.edge('None', start_state)
        else:
            if state!=start_state:
                graph.node(state, label=state, shape='doublecircle' if state_data['isTerminatingState'] else 'circle')

    # Add edges for transitions
    for state, state_data in nfa_data.items():
        if state != 'startingState':
            for symbol, next_state in state_data.items():
                if symbol not in ['isTerminatingState', 'epsilon']:
                    graph.edge(state, next_state, label=symbol)
            for epsilon_move in state_data.get('epsilon', []):
                graph.edge(state, epsilon_move, label='ε')

    # Save the graph to a file
    graph.render('nfa_graph', format='png', cleanup=True)

    print("NFA graph generated successfully.")

In [161]:
def part1(reg):
  #reg=reg#"a|s+t?[a-ce-r]"#"(a*b)(b?a+)"#"(a*?)*"#"(a*)*"#"(a|b)*a[ab]?"#"(a+a+)+b"#"(a*b*)([a-b]*)"##"[a-c]*" #"(a*b)(b?a+)"  #"([)" #"[a-z_][a-z0-9_]*[!?]?" #"(a|b)*bc+" #"((00)|1)*1(0|1)" #"(a*)*" #"ab(b|c)*d+" #"aa+b*b" #"(a++"
      #"a|bc1+3d4(df2)(y+2)"
  #validate the regular expression
  if not validate_reg(reg):
      print("invalid regular expression")
      exit()
  #validate the brackets
  if validate_brackets(reg)=="unbalanced parentheses":
      print("unbalanced parentheses")
      exit()
  print("\n======")
  print("after adding anding the regular expression")
  regex_anded=regex_anding(reg)
  print(regex_anded)
  print("\n======")
  print("after concating the range in one string")
  correct_regex_with_range=concat_range(regex_anded)
  if correct_regex_with_range==["%"]:
      print("invalid range expression")
      exit()
  print(correct_regex_with_range)
  print("\n======")
  print("after removing extra anding")
  correct_regex=remove_extra_anding(correct_regex_with_range)
  print(correct_regex)
  print("\n======")
  postfix=infix_to_postfix(correct_regex)
  if postfix=="unbalanced parentheses":
      print("unbalanced parentheses")
      exit()
  print("after converting the regular expression to postfix")#['[a-z_]', '×', '[a-z0-9_]', ']*', '×', '[!?]', ']?']
  print(postfix)
  print("\n======")

  nfa,all_states=postfix_to_nfa(postfix)
  print("start state ")
  print(nfa.start.label)
  print("accept state")
  print(nfa.accept.label)
  print("\n======")
  nfa_dict=create_dict_for_jason(nfa,all_states)
  print("NFA in json format")
  # print(nfa_dict)
  create_json(nfa_dict)
  # Example usage
  draw_nfa("NFA.json")

In [162]:
import json
from queue import Queue
from automathon import DFA



def DFADrawer(DFADiagram, alphabet, drawingName="DFA"):
    print("MinimizedDFA",DFADiagram)
    # Extract states
    states = set(DFADiagram.keys()) - {'startingState'}
    # Extract initial state
    initialState = DFADiagram['startingState']

    # Extract final states
    finalStates = {k for k, v in DFADiagram.items() if k != 'startingState' and v['isTerminatingState']}

    # Extract transitions
    transitions = {k: {key: value for key, value in v.items() if key not in ['isTerminatingState', 'isStartingState']}
                   for k, v in DFADiagram.items() if k != 'startingState'}

    # Create DFA object
    automata2 = DFA(states, alphabet, transitions, initialState, finalStates)

    # View DFA
    automata2.view(drawingName)



def stateEquality(stateList1, stateList2):
    # Checks if state is unique or not
    if len(stateList1) != len(stateList2):
        return False

    # Create sets of state names for fast membership testing
    state_names1 = {list(state.keys())[0] for state in stateList1}
    state_names2 = {list(state.keys())[0] for state in stateList2}

    # Check if the sets of state names are equal
    return state_names1 == state_names2

def checkUnique(lst, elem):
    # check if element is unique
    return lst.count(elem) == 0

def epsilonClosure(state, allStates):

    newStateList = []  # List to store states in the epsilon closure
    findEpsilon = Queue(maxsize=0)  # Queue to perform BFS for epsilon transitions

    # Extract state key and value
    key, value = list(state.keys())[0], list(state.values())[0]
    findEpsilon.put(value)
    newStateList.append(state)

    while not findEpsilon.empty():
        # Walk through each state in epsilon of current state if any epsilon transitions exist
        current_state = findEpsilon.get()
        if current_state==None:
            continue
        if "epsilon" in current_state:
            for i in current_state['epsilon']:
                # Check uniqueness before adding to the queue and newStateList
                if checkUnique(newStateList, {i: allStates.get(i)}):
                    findEpsilon.put(allStates.get(i))
                    newStateList.append({i: allStates.get(i)})

    # Return all states in this epsilon closure
    return newStateList
def move(state, char, allStates):

    stateList = []  # List to store states reached by the move operation
    print('state is: ', state)  # Print the current state

    # Loop through the state's key-value pairs
    for key, value in state.items():
        # Skip 'isTerminatingState' key
        if key == 'isTerminatingState':
            continue

        # Check if the input character leads to any next states
        print(value)
        if char in value:
            nextState = value[char]
            for i in nextState:
                # Compute the epsilon closure for each next state and add to stateList
                stateList.extend(epsilonClosure({i: allStates.get(i)}, allStates))

    return stateList

stateCounter = 0
def stateMaker(states, alphabet):

    global stateCounter  # Global counter for state names
    new_state = {}  # Dictionary to store the new state

    # Create state name
    new_state["stateList"] = states
    new_state["bigStateName"] = "S" + str(stateCounter)
    stateCounter += 1

    # Initialize transitions for each character in the alphabet
    for char in alphabet:
        new_state[char] = ""

    return new_state


def makeDFA(states):

    queue = Queue(maxsize=0)  # Queue for state exploration
    queue.put(states[0])  # Add starting state to the queue

    print('states is: ', states)  # Print the initial big state list

    # Explore states in the DFA
    while not queue.empty():
        curr_state = queue.get()  # Get the current big state

        curr_state_list = curr_state["stateList"]  # Extract list of states from the big state
        for char in alphabet:
            new_state_list = []  # List to store states reached by transition on character 'char'

            # Compute the move operation for each state in the current big state
            for state in curr_state_list:
                new_states = move(state, char, allStates)
                new_state_list.extend([new_state for new_state in new_states if new_state not in new_state_list])

            # Create a new big state from the list of reached states
            new_state_list = stateMaker(new_state_list, alphabet)

            # Skip if the new big state is empty
            if len(new_state_list["stateList"]) == 0:
                continue

            # Check if the new big state already exists in the big state list
            state_exists = False
            connect_to_state = new_state_list
            for s in states:
                if stateEquality(s["stateList"], new_state_list["stateList"]):
                    state_exists = True
                    connect_to_state = s
                    break

            # Update transitions of the current big state
            curr_state[char] = connect_to_state["bigStateName"]

            # Add the new big state to the big state list if it doesn't already exist
            if not state_exists:
                states.append(new_state_list)
                print('new_state_list is: ', new_state_list)
                queue.put(new_state_list)


def format_dfa(big_state_list):
    counter = 0
    state_names_dict = {}

    # Sanitize state names and create a mapping
    for state in big_state_list:
        old_state_name = state['bigStateName']
        new_state_name = "S" + str(counter)
        state_names_dict[old_state_name] = new_state_name
        state['bigStateName'] = new_state_name
        counter += 1

    # Mark terminal states
    for state in big_state_list:
        print("state['stateList']", state['stateList'])
        is_terminal = any(v['isTerminatingState']  for s in state['stateList'] for k, v in s.items())
        state['isTerminatingState'] = is_terminal

    # Remove stateList and make stateName the key in the final dictionary
    dfa_dict = {'startingState': None}
    for state in big_state_list:
        if state.get('isStartingState') and state['isStartingState']:
            dfa_dict['startingState'] = state['bigStateName']
            break
    for state in big_state_list:
        del state['stateList']
        dfa_dict[state['bigStateName']] = state
        del state['bigStateName']

    # Update all old references to new state names
    for state_name, state in dfa_dict.items():
        if state_name != 'startingState':
            for char in alphabet:
                if char == 'isTerminatingState':
                    continue

                if char in state and len(state[char]) != 0:
                    state[char] = state_names_dict[state[char]]
                # remove unnecessary key characters
                if char in state and len(state[char]) == 0:
                    del state[char]

    return dfa_dict

def get_state_from_name(state_name):

    global DFADiagram
    for k, v in DFADiagram.items():
        if k == state_name:
            return {k: v}
    return None

def get_new_state(state, char):

    for k, v in state.items():
        if k not in ('isStartingState', 'isTerminatingState') and char in v:
            state_name = v[char]
            return get_state_from_name(state_name)

    return None


def get_state_group(state, state_groups):

    for group in state_groups:
        if state is None:
            break
        for k, v in state.items():
            if k not in ('isStartingState', 'isTerminatingState') and k in group["stateNames"]:
                return group
    return None
def get_state_name(state):

    for key, value in state.items():
        if key not in ('isStartingState', 'isTerminatingState'):
            return key


def check_other_group_members(new_state_group, state_group, current_state_groups, character):

    for state in state_group["states"]:
        new_state = get_new_state(state, character)
        if new_state is None:
            return False  # If there is no state to transition to, it's not in the same group

        # Get the state group of the new state
        vnew_state_group = get_state_group(new_state, current_state_groups)
        if new_state_group != vnew_state_group:
            return False

    return True


def check_other_groups(state, original_state_group, current_state_groups, alphabet):
    for state_group in current_state_groups:
        # Skip the original state group
        if state_group == original_state_group:
            continue

        # Take any state from the group
        if len(state_group["states"]) == 0:
            continue

        the_other_state = state_group["states"][0]
        flag = True

        # Check if the current state and any state from the other group have the same transitions
        for char in alphabet:
            new_state = get_new_state(state, char)
            the_other_state_new_state = get_new_state(the_other_state, char)

            #print("new_state", list(new_state.values())[0]["isTerminatingState"])
            if state != None and  list(state.values())[0]["isTerminatingState"] :
                    flag = False
                    break
            if the_other_state != None and  list(the_other_state.values())[0]["isTerminatingState"] :
                    flag = False
                    break
            if new_state != the_other_state_new_state :
                flag = False
                break

        # If all transitions match, return the other state group
        if flag:
            return state_group

    # If no matching state group is found, return None
    return None

    return None
def minDFA(state_groups, alphabet):
    if len(state_groups) == 1:
        return state_groups
    old_state_group_size = -1
    # Continue until no further merges can be made
    while old_state_group_size != len(state_groups):

        # Create a copy of state_groups to iterate over
        curr_state_groups = state_groups.copy()
        # Loop through each state group
        for state_group in curr_state_groups:
            # If the state group has only one state, check if it can be merged with another state group
            if len(state_group["states"]) == 1:
                state = state_group["states"][0]
                potential_merge_group = check_other_groups(state, state_group, state_groups, alphabet)
                if potential_merge_group!=None:
                    # Merge the state group with the potential merge group
                    potential_merge_group["stateNames"].append(get_state_name(state))
                    potential_merge_group["states"].append(state)
                    state_group["stateNames"].remove(get_state_name(state))
                    state_group["states"].remove(state)
                    # Remove the empty state group
                    state_groups.remove(state_group)
                continue
            # Loop through each state in the state group and each character in the alphabet
            for state in state_group["states"]:
                for char in alphabet:
                    # Get the new state resulting from the transition on the current character
                    new_state = get_new_state(state, char)
                    if new_state is None and not list(state.values())[0]["isTerminatingState"] :
                        continue
                    # Get the state group of the new state
                    new_state_group = get_state_group(new_state, state_groups)
                    # If the new state group is different from the current state group, check for equivalence
                    if new_state_group != state_group:
                        if not check_other_group_members(new_state_group, state_group, curr_state_groups, char):
                            # If the rest of the group members do not transition to the same group, create a new group
                            potential_merge_group = check_other_groups(state, state_group, state_groups, alphabet)
                            if potential_merge_group!=None:
                                # Merge the state group with the potential merge group
                                potential_merge_group["stateNames"].append(get_state_name(state))
                                potential_merge_group["states"].append(state)
                                state_group["stateNames"].remove(get_state_name(state))
                                state_group["states"].remove(state)

                            else:
                                # Create a new group for the state
                                new_group_of_state = {"stateNames": [get_state_name(state)], "states": [state]}
                                state_groups.append(new_group_of_state)
                                state_group["stateNames"].remove(get_state_name(state))
                                state_group["states"].remove(state)
                            break
        old_state_group_size = len(curr_state_groups)
    return state_groups



def format_minimised_dfa(minimised_dfa, alphabet):
    # Name all the state groups
    for i, state_group in enumerate(minimised_dfa):
        state_group["stateGroupName"] = 'S' + str(i)

    final_states_dict = {}
    for state_group in minimised_dfa:
        final_state_form = {}
        for state in state_group["states"]:
            state_info = list(state.values())[0]
            for k, v in state_info.items():
                if k == 'isStartingState':
                    final_state_form["isStartingState"] = True
                if k == 'isTerminatingState':
                    final_state_form["isTerminatingState"] = v

        new_name = state_group["stateGroupName"]
        for char in alphabet:
            first_state = list(state_group["states"][0].values())[0]
            if char not in first_state.keys():
                continue
            new_state = get_new_state(state_group["states"][0], char)
            new_state_group = get_state_group(new_state, minimised_dfa)
            if new_state_group is None:
                continue
            final_state_form[char] = new_state_group["stateGroupName"]

        final_states_dict[new_name] = final_state_form

    very_final_state = {}
    print("final_states_dict ",final_states_dict)
    for name, state_info in final_states_dict.items():
        if 'isStartingState' in state_info.keys():
            very_final_state['startingState'] = name
            break
    very_final_state.update(final_states_dict)

    # Print the formatted DFA for debugging
    print('Printing the formatted DFA:')
    for name, state_info in very_final_state.items():
        print(name, state_info)

    return very_final_state


In [163]:
DFADiagram = dict()
alphabet= set()
allStates =None

def part2():

  global allStates
  global alphabet
  global DFADiagram
  with open("NFA.json", "r") as inputFile:
    allStates= json.load(inputFile)


  # get alphabet
  for state in allStates:
      if state != 'startingState':
          for key, value in allStates[state].items():
              if key != 'isTerminatingState':
                  if type(value) != list:
                      value = [value]
                      allStates[state][key] = value
              if key != 'isTerminatingState' and key != 'epsilon':
                      alphabet.add(key)


  print('alphabet is: ', alphabet)
  #part2 main

  startingState = allStates.get("startingState")

  # get epsilon closure of starting state
  initState = epsilonClosure(
      {startingState: allStates.get(startingState)}, allStates)
  initState = stateMaker(initState, alphabet) #TODO: check if this is correct
  # initState by this point contains the big State
  initState["isStartingState"] = True
  bigStateList = [initState]

  makeDFA(bigStateList)
  DFADiagram = format_dfa(bigStateList)
  DFADrawer(DFADiagram, alphabet, "DFA")

  print('printing after DFA Formatter: ')
  for key, value in DFADiagram.items():
      print(key, value)


  # let state group be of form {states: {{S1:bla bla}, {S2:bla bla}}, stateNames: ["S1", "S2"]}

  # minimization
  nonTerminalStates = {"states": [], "stateNames": []}
  TerminalStates = {"states": [], "stateNames": []}


  for k, v in DFADiagram.items():
      if k != 'startingState' and v['isTerminatingState'] == True:
          TerminalStates["states"].append({k: v})
          TerminalStates["stateNames"].append(k)
      else:
          if k != 'startingState':
              nonTerminalStates["states"].append({k: v})
              nonTerminalStates["stateNames"].append(k)


  # keda we have the starting 2 groups
  StateGroups = []
  if (len(nonTerminalStates["stateNames"]) == 0):
      StateGroups = [TerminalStates]
  elif (len(TerminalStates["stateNames"]) == 0):
      StateGroups = [nonTerminalStates]
  elif (len(TerminalStates["stateNames"]) != 0 and len(nonTerminalStates["stateNames"]) != 0):
      StateGroups = [nonTerminalStates, TerminalStates]


  minimisedDFA = minDFA(StateGroups, alphabet)

  print('printing minimised DFA: ', minimisedDFA)

  format_minDFA = format_minimised_dfa(minimisedDFA, alphabet)

  for state in format_minDFA:
      print(state)
  with open("MinimizedDFA.json", "w") as outfile:
      json.dump(format_minDFA, outfile, indent=4)
  DFADrawer(format_minDFA, alphabet, "MinimizedDFA")

In [164]:
part1("(a+?a+?)+?b")#(a*b+[a-z](C?))("(a|b)*a[ab]?")#("(a+a+)+b")#("(a+?a+?)+?b")#("(a*b*)([a-b]*)")#("(a*b)(b?a+)")#("(a*)*")#("(a*?)*")#("a+b*a")#("kam*(o|ou)la?")#("ab(b|c)*d+")#("a+|b+")
part2()


after adding anding the regular expression
['(', 'a', '+', '?', '×', 'a', '+', '?', '×', ')', '+', '?', '×', 'b']

after concating the range in one string
['(', 'a', '+', '?', '×', 'a', '+', '?', '×', ')', '+', '?', '×', 'b']

after removing extra anding
['(', 'a', '+', '?', '×', 'a', '+', '?', ')', '+', '?', '×', 'b']

after converting the regular expression to postfix
['a', '+', '?', 'a', '+', '?', '×', '+', '?', 'b', '×']

start state 
14
accept state
17

NFA in json format
json file created successfully
NFA graph generated successfully.
alphabet is:  {'a', 'b'}
states is:  [{'stateList': [{'S14': {'isTerminatingState': False, 'epsilon': ['S12', 'S15']}}, {'S12': {'isTerminatingState': False, 'epsilon': ['S4']}}, {'S15': {'isTerminatingState': False, 'epsilon': ['S16']}}, {'S4': {'isTerminatingState': False, 'epsilon': ['S2', 'S5']}}, {'S16': {'isTerminatingState': False, 'b': ['S17']}}, {'S2': {'isTerminatingState': False, 'epsilon': ['S0']}}, {'S5': {'isTerminatingState': False, 